# Testing Qwen2.5-Coder 7B on Ollama with xpander.ai’s Function Calling

In [ ]:
pip install ollama; pip install xpander-sdk


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from xpander_sdk import XpanderClient
import ollama
## Load environment variables
from dotenv import load_dotenv
import os
load_dotenv()
xpanderAPIKey = os.environ.get("XPANDER_API_KEY","")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")
ollama_client = ollama.Client()
xpander_client = XpanderClient(api_key=xpanderAPIKey)
cloud_function_agent = xpander_client.agents.get(agent_id=xpanderAgentID)

In [2]:
# Verify that you have ollama installed locally and can get responses before moving on
response = ollama_client.chat(model='qwen2.5-coder:14b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue during the day because of a phenomenon called Rayleigh scattering. This occurs when sunlight enters Earth's atmosphere and encounters gas molecules and tiny particles in the air. Sunlight is made up of different colors, each with its own wavelength. Blue light has shorter wavelengths and higher energy compared to other colors like red or orange.

When sunlight reaches the Earth's atmosphere, the blue light scatters in all directions more than the other colors because it travels as smaller, shorter waves. This scattering process happens more frequently for blue light than for longer-wavelength light such as red. As a result, we see predominantly blue light when we look up at the sky during daylight hours.

This is why the sky often appears blue on clear days, but can take on other colors like orange or pink during sunrise and sunset, when the sun's rays have to pass through more of the Earth's atmosphere. The additional path length causes more scattering of the shor

In [12]:
personal_ai_agent = xpander_client.agents.get(agent_id="1724ddf7-4246-4bdd-b39f-65314e84e662")

# Define the local functions

In [25]:
import json
import xml.etree.ElementTree as ET
import os
import csv
from io import StringIO

def is_within_current_directory(file_path):
    """
    Validates that the given file path is within the current directory.

    Args:
        file_path (str): The file path to check.

    Returns:
        bool: True if the file path is within the current directory, False otherwise.
    """
    base_path = os.path.abspath(".")
    absolute_path = os.path.abspath(file_path)
    return absolute_path.startswith(base_path)

def read_file(file_path, output_format=None):
    """
    Reads the content of a file and optionally converts it to the specified format.

    Args:
        file_path (str): The path to the file to read.
        output_format (str, optional): The format to convert the file content to ('json', 'text', 'xml', 'csv').

    Returns:
        str or list or dict: The content of the file in the requested format or an error message.
    """
    if not is_within_current_directory(file_path):
        return {"error": "Access to files outside the current directory is not allowed."}

    if not os.path.exists(file_path):
        return {"error": f"File not found: {file_path}"}
    
    try:
        # Read file content
        if output_format == "csv":
            with open(file_path, "r", newline="") as file:
                reader = csv.reader(file)
                return [row for row in reader]
        else:
            with open(file_path, "r") as file:
                content = file.read()
        
        # Handle output format conversion
        if output_format == "json":
            try:
                return json.loads(content)
            except json.JSONDecodeError:
                return {"error": "Failed to convert file content to JSON."}
        elif output_format == "xml":
            try:
                root = ET.fromstring(content)
                return ET.tostring(root, encoding='unicode')
            except ET.ParseError:
                return {"error": "Failed to convert file content to XML."}
        elif output_format == "text" or output_format is None:
            return content
        else:
            return {"error": f"Unsupported output format: {output_format}"}
    except Exception as e:
        return {"error": f"An error occurred while reading the file: {str(e)}"}

def write_file(file_path, content, fileType):
    """
    Writes content to a file, converting it to the specified format before saving.

    Args:
        file_path (str): The path to the file to write.
        content (str or list): The content to write to the file. For 'csv', content can be a string or a list of lists.
        fileType (str): The format in which to save the file ('json', 'txt', 'csv', 'xml').

    Returns:
        dict: A success or error message.
    """
    def is_within_current_directory(file_path):
        base_path = os.path.abspath(".")
        absolute_path = os.path.abspath(file_path)
        return absolute_path.startswith(base_path)

    if not is_within_current_directory(file_path):
        return {"error": "Access to files outside the current directory is not allowed."}

    try:
        # Handle content format conversion
        if fileType == "json":
            try:
                if isinstance(content, str):
                    parsed_content = json.loads(content)
                else:
                    parsed_content = content
                with open(file_path, "w") as file:
                    json.dump(parsed_content, file, indent=2)
            except json.JSONDecodeError:
                return {"error": "Invalid JSON content provided."}
        elif fileType == "xml":
            try:
                root = ET.Element("root")
                ET.SubElement(root, "content").text = str(content)
                tree = ET.ElementTree(root)
                tree.write(file_path, encoding="unicode", xml_declaration=True)
            except Exception as e:
                return {"error": f"Failed to write XML file: {str(e)}"}
        elif fileType == "csv":
            try:
                # Handle if content is a string
                if isinstance(content, str):
                    content = list(csv.reader(StringIO(content)))
                if not isinstance(content, list) or not all(isinstance(row, list) for row in content):
                    return {"error": "Invalid CSV content provided. It must be a string or a list of lists."}
                with open(file_path, "w", newline="") as file:
                    writer = csv.writer(file)
                    writer.writerows(content)
            except Exception as e:
                return {"error": f"Failed to process CSV content: {str(e)}"}
        elif fileType == "txt" or fileType == "text":
            with open(file_path, "w") as file:
                file.write(content)
        else:
            return {"error": f"Unsupported file format: {fileType}"}
        
        return {"success": f"File written successfully to {file_path} in {fileType} format."}
    except Exception as e:
        return {"error": f"Failed to write file: {str(e)}"}

    
# Tests

# Test 1: Write plain text
print(write_file("test.txt", "Hello, this is a test file.", "txt"))

# Test 2: Write JSON
json_content = {"name": "John Doe", "age": 30, "email": "john.doe@example.com"}
print(write_file("test.json", json_content, "json"))

# Test 3: Write CSV
csv_content = [["Name", "Age", "Email"], ["John Doe", 30, "john.doe@example.com"], ["Jane Smith", 25, "jane.smith@example.com"]]
print(write_file("test.csv", csv_content, "csv"))

# Test 4: Write XML
xml_content = "Sample XML content"
print(write_file("test.xml", xml_content, "xml"))


# Test 1: Read plain text
print(read_file("test.txt", "text"))

# Test 2: Read JSON
print(read_file("test.json", "json"))

# Test 3: Read CSV
print(read_file("test.csv", "csv"))

# Test 4: Read XML
print(read_file("test.xml", "xml"))

# Test 5: Read unsupported format
print(read_file("test.txt", "unsupported_format"))

# Test 6: Attempt to read a non-existent file
print(read_file("non_existent.txt", "text"))

# CSV String Input Example
csv_string = "Name,Age\nAlice,25\nBob,30"
result = write_file("example.csv", csv_string, "csv")
print(result)

{'success': 'File written successfully to test.txt in txt format.'}
{'success': 'File written successfully to test.json in json format.'}
{'success': 'File written successfully to test.csv in csv format.'}
{'success': 'File written successfully to test.xml in xml format.'}
Hello, this is a test file.
{'name': 'John Doe', 'age': 30, 'email': 'john.doe@example.com'}
[['Name', 'Age', 'Email'], ['John Doe', '30', 'john.doe@example.com'], ['Jane Smith', '25', 'jane.smith@example.com']]
<root><content>Sample XML content</content></root>
{'error': 'Unsupported output format: unsupported_format'}
{'error': 'File not found: non_existent.txt'}
{'success': 'File written successfully to example.csv in csv format.'}


In [27]:
localTools = [
    {
        "type": "function",
        "function": {
            "name": "read-file",
            "description": "Reads the content of a file from the local system within the current working directory and returns its content. The tool is restricted to operate only within the current directory and cannot access files outside this location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "bodyParams": {
                        "type": "object",
                        "properties": {
                            "filePath": {
                                "type": "string",
                                "description": "The name of the file to be read (must be in the current working directory). Absolute or relative paths pointing outside the current directory are not allowed.",
                                "examples": ["file.txt", "data.csv"]
                            },
                            "outputFormat": {
                                "type": "string",
                                "description": "The format to convert the file's content into (e.g., 'json', 'text', 'xml').",
                                "examples": ["json", "text", "xml"]
                            }
                        },
                        "required": ["filePath"],
                        "additionalProperties": False
                    }
                },
                "required": ["bodyParams"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "write-file",
            "description": "Writes content to a file in the current working directory. The tool cannot write to locations outside this directory, ensuring restricted access. Only the file name should be provided, without paths.",
            "parameters": {
                "type": "object",
                "properties": {
                    "bodyParams": {
                        "type": "object",
                        "properties": {
                            "fileName": {
                                "type": "string",
                                "description": "The name of the file to be written (must be in the current working directory). Absolute or relative paths pointing outside the current directory are not allowed.",
                                "examples": ["output.json", "log.txt"]
                            },
                            "fileContent": {
                                "type": "string",
                                "description": "The content to write to the file.",
                                "examples": [
                                    '{"name": "John Doe", "age": 30}',
                                    "Hello, world!",
                                    "Name,Age\nAlice,25\nBob,30"
                                ]
                            },
                            "fileType": {
                                "type": "string",
                                "description": "The file format to save the content in (e.g., 'json', 'txt', 'csv', 'xml').",
                                "examples": ["json", "txt", "csv", "xml"]
                            }
                        },
                        "required": ["filePath", "fileContent", "fileType"],
                        "additionalProperties": False
                    }
                },
                "required": ["bodyParams"],
                "additionalProperties": False
            }
        }
    }
]

personal_ai_agent.add_local_tools(localTools)

personal_ai_agent.get_tools()

[{'type': 'function',
  'function': {'name': 'PgTomorrowWeatherForecastGetForecastByLocation',
   'description': 'This operation provides the weather forecast for a specified location, such as Tel Aviv. Use this when you need real-time weather updates or forecasts for planning events or activities, making it essential for applications focused on travel, outdoor events, or local services. IMPORTANT! Ensure to use body_params, query_params, path_params. These are crucial for correct function calling!',
   'parameters': {'type': 'object',
    'properties': {'queryParams': {'type': 'object',
      'properties': {},
      'required': []},
     'pathParams': {'type': 'object', 'properties': {}, 'required': []},
     'bodyParams': {'type': 'object',
      'properties': {'inputTask': {'type': 'string',
        'description': 'input user sub task based on the input query'}},
      'required': []}},
    'required': ['query_params', 'path_params', 'body_params']}}},
 {'type': 'function',
  'funct

In [ ]:
memory = []
memory.append({"role": "system", "content": "You are a helpful assistant with function calling and tool access. Pay attention to use write-file and read-file carefuly with bodyParams, fileName, fileContent and fileType . Filetype can be only txt, csv, json or xml. .you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## in your answer"})
print(memory[-1])
memory.append({"role": "user", "content": "Create four hello world files , one CSV , one JSON and one XML and one text. Generate sample data for each. Example CSV data MUST BE in string 'Name,Age\nAlice,25\nBob,30"})
print(memory[-1])
number_of_calls = 1
from xpander_sdk import LLMProvider, ToolCallType

while True:
    llm_response = ollama_client.chat(
        model="qwen2.5-coder:14b",
        messages=memory,
        tools=personal_ai_agent.get_tools()    
    )
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    print(memory[-1])

    memory.append(llm_response['message'])
    print(memory[-1])
    if llm_response['message'].get('tool_calls'):
        print("llm_response")
        print(llm_response['message'].get('tool_calls'))
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response,llm_provider=LLMProvider.OLLAMA)
        for tool_to_run in tools_to_run:
            if tool_to_run.type == ToolCallType.LOCAL:
                function_name = tool_to_run.name
                # Extracting parameters with defaults
                body_params = tool_to_run.payload.get('bodyParams', {})
                fileName = body_params.get('fileName', 'default_file.txt')
                fileContent = body_params.get('fileContent', '')
                fileType = body_params.get('fileType', 'txt')
                print(function_name, fileName,fileContent,fileType)
                if("write-file" in function_name):
                    result = write_file(file_path=fileName, content=fileContent, fileType=fileType)
                memory.append({"role": "tool", "content": json.dumps(result), "tool_call_id": function_name})
            else:
                tool_response = personal_ai_agent.run_tool(tool_calls=tools_to_run)
                memory.append({"role": "tool", "content": tool_response.result, "tool_call_id": tool_response.tool_call_id})
            print(memory[-1])
    if (llm_response['message'].get('content')):
        if "##FINAL ANSWER##" in llm_response['message']['content']:
            break
    number_of_calls += 1
print(llm_response['message']['content'])

{'role': 'system', 'content': 'You are a helpful assistant with function calling and tool access. Pay attention to use write-file and read-file carefuly with bodyParams, fileName, fileContent and fileType . Filetype can be only txt, csv, json or xml. .you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## in your answer'}
{'role': 'user', 'content': "Create four hello world files , one CSV , one JSON and one XML and one text. Generate sample data for each. Example CSV data MUST BE in string 'Name,Age\nAlice,25\nBob,30"}
{'role': 'assistant', 'content': 'Step number: 1'}
{'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'xpLocal_write-file', 'arguments': {'bodyParams': {'fileContent': 'Name,Age\nAlice,25\nBob,30', 'fileName': 'hello_world.csv', 'fileType': 'csv'}}}}, {'function': {'name': 'xpLocal_write-file', 'arguments': {'bodyParams': {'fileContent': '{"greeting": "Hello, world!", "data": [{"name": "Alice", "age": 25}, {"name": 